In [ ]:
%%bash
git clone https://github.com/huggingface/transformers
cd transformers
pip install .

Processing /content/transformers
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for transformers: filename=transformers-4.41.0.dev0-py3-none-any.whl size=9046235 sha256=2b7f6a16adaa8bb9a63081ef6561f822b482c19707fcfcd4abe22163ed5da779
  Stored in directory: /tmp/pip-ephem-wheel-cache-zt9sbn1y/wheels/7c/35/80/e946b22a081210c6642e607ed65b2a5b9a4d9259695ee2caf5
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.40.1
    Uninstalling transformers-4.40.1:
      Successfully uninstalled transformers-4.40.1


Cloning into 'transformers'...


In [ ]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 26.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 39.2 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3


In [ ]:
!pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 1.9 MB/s eta 0:00:00


In [ ]:
!mkdir models/

In [1]:
import pandas as pd
import os
import numpy as np
import string

In [2]:
# Считаем тренировочные данные.

df_train = pd.read_csv('/content/drive/MyDrive/Диплом_2024/data/therapy_train_true.csv')

df_train.head()

,text
0,Московский государственный медико-стоматологич...
1,Башкирский Государственный Медицинский Универс...
2,Министерство здравоохранения Республики Белару...
3,\nПаспортная часть\n\nФИО: \nВозраст: 29 лет\n...
4,\nИстория болезни.\nФамилия: \n Имя: \nОтчест...


In [3]:
# Считаем тестовые данные.

df_test = pd.read_csv('/content/drive/MyDrive/Диплом_2024/data/test_set.csv')

df_test.head()

,text
0,Министерство здравоохранения Российской Феде...
1,"Жалобы на слабость, отдышку, изнуряющий кашель..."
2,Диагноз: Пневмония в правой нижней доле. ДН II...
3,Основные жалобы - на периодический кашель с не...
4,Клинический диагноз: Внебольничная правосторон...


In [4]:
print(df_train.shape)

print(df_test.shape)

(65, 1)
(9, 1)


In [5]:
pd.set_option('display.max_colwidth', None)

df_train.loc[0]

text    Московский государственный медико-стоматологический университет\nкафедра пропедевтики внутренних болезней стоматологического факультета\n(заведующий кафедрой  - заслуженный деятель науки РФ, профессор Токмачев Юрий Константинович) \n\n\n\n\n\n\n\n\n\n\nИСТОРИЯ БОЛЕЗНИ\nБольного Коновалова А.Д., 65 лет\n5 терапевтическое отделение, палата 102\n\n\n\n\n\n\n\n\n\nКуратор студентка III курса\n 30 группы дневного \nстоматологического факультета\nКоваленко Александры Валериевны\n\n\n\nПреподаватель: Пихлак А.Э.\n\n\n\n\n\n\n\n\nМосква, 2000\n\n\n\nПаспортные данные\n\nФ.И.О.   Коновалов  Алексей Дмитриевич \nГод рождения 1935\nПрофессия:  на пенсии, ранее работал строителем, литейщиком пластмасс\nМесто жительства:  г. Москва, Бескуднический бульвар, 18-2-14\nДата поступления в клинику 13.11.00 \n\nЖалобы при поступлении\n\nна  внезапно возникшее чувство резкой нехватки воздуха, постепенно нараставшее и продолжавшееся до 30 минут, не проходящее при использовании ингалятора. В разгар п

In [6]:
# Уберем лишние знаки и цифры из текста трейн.

df_train = df_train.replace(r'\n', ' ', regex=True)

df_train = df_train.replace(r'\t', ' ', regex=True)

df_train['text'] = df_train['text'].str.lower()

df_train['text'] = df_train['text'].str\
                                   .replace('[{}]'.format(string.punctuation), '', regex=True)

df_train['text'] = df_train['text'].str.replace(' {2,}', ' ', regex=True)

from string import digits

remove_digits = str.maketrans('', '', digits)

df_train['text'] = df_train['text'].str.translate(remove_digits)


df_train.loc[0]

text    московский государственный медикостоматологический университет кафедра пропедевтики внутренних болезней стоматологического факультета заведующий кафедрой заслуженный деятель науки рф профессор токмачев юрий константинович история болезни больного коновалова ад  лет  терапевтическое отделение палата  куратор студентка iii курса  группы дневного стоматологического факультета коваленко александры валериевны преподаватель пихлак аэ москва  паспортные данные фио коновалов алексей дмитриевич год рождения  профессия на пенсии ранее работал строителем литейщиком пластмасс место жительства г москва бескуднический бульвар  дата поступления в клинику  жалобы при поступлении на внезапно возникшее чувство резкой нехватки воздуха постепенно нараставшее и продолжавшееся до  минут не проходящее при использовании ингалятора в разгар приступа появлялся кашель с отделением небольшого количества слизистой мокроты течение приступа облегчалось принятием сидячего положения с упором рук на колени прис

In [7]:
# Уберем лишние знаки и цифры из текста тест.

df_test = df_test.replace(r'\n', ' ', regex=True)

df_test = df_test.replace(r'\t', ' ', regex=True)

df_test['text'] = df_test['text'].str.lower()

df_test['text'] = df_test['text'].str\
                                 .replace('[{}]'.format(string.punctuation), '', regex=True)

df_train['text'] = df_train['text'].str.replace(' {2,}', ' ', regex=True)


remove_digits = str.maketrans('', '', digits)

df_test['text'] = df_test['text'].str.translate(remove_digits)


df_test.iloc[0]

text      министерство здравоохранения российской федерации  алтайский государственный медицинский университет кафедра пропедевтики внутренних болезней зав кафедрой проф            академическая история болезни          больной куратор студентка  группы iii курса лечебного факультета время курации  –  г преподаватель           паспортная часть   фио   возраст  лет   место работы центр занятости населения  место жительства  дата поступления в клинику  г  диагноз пневмония в правой нижней доле дн ii острый бронхит pneumonia crouposae dextri lobi inferioris insufficientia respiratoria gradus ii                   status praesens subjectivus  жалобы больного на момент осмотра редкий сухой кашель одышка при физической нагрузке лёгкая слабость недомогание головокружение жалобы на момент поступления редкий сухой кашель одышка при физической нагрузке слабость недомогание распирающие головные боли головокружение шум в ушах закладывает уши при повышении и снижении ад боли за грудиной давящего хар

In [ ]:
with open('train.txt', 'a') as f:
    df_string = df_train.to_string(header=False, index=False)
    f.write(df_string)

In [ ]:
with open('valid.txt', 'a') as f:
    df_string = df_test.to_string(header=False, index=False)
    f.write(df_string)

In [ ]:
!wget https://raw.githubusercontent.com/huggingface/transformers/main/examples/pytorch/language-modeling/run_clm.py

--2024-05-06 13:57:51--  https://raw.githubusercontent.com/huggingface/transformers/main/examples/pytorch/language-modeling/run_clm.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 28908 (28K) [text/plain]
Saving to: ‘run_clm.py’

run_clm.py          100%[===================>]  28.23K  --.-KB/s    in 0s      

2024-05-06 13:57:51 (125 MB/s) - ‘run_clm.py’ saved [28908/28908]



In [ ]:
!pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.4/302.4 kB 7.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [ ]:
!python run_clm.py \
    --model_name_or_path sberbank-ai/rugpt3small_based_on_gpt2 \
    --train_file train.txt \
    --validation_file valid.txt \
    --num_train_epochs 35\
    --per_device_train_batch_size 16 \
    --per_device_eval_batch_size 16 \
    --block_size 64 \
    --dataset_config_name plain_text \
    --do_train \
    --do_eval \
    --save_total_limit 1\
    --output_dir models/therapy_gpt

2024-05-06 13:59:26.296464: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-06 13:59:26.296528: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-06 13:59:26.409604: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-06 13:59:26.479206: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-06 13:59:29.087827: W tensorflow/comp

In [ ]:
import numpy as np
import torch

In [ ]:
np.random.seed(42)
torch.manual_seed(42)

In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

In [ ]:
tok = GPT2Tokenizer.from_pretrained("models/therapy_gpt")

In [ ]:
model = GPT2LMHeadModel.from_pretrained("models/therapy_gpt")

In [ ]:
model.cuda()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50264, 768)
    (wpe): Embedding(2048, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50264, bias=False)
)

In [ ]:
text = "хрипы в"
inpt = tok.encode(text, return_tensors="pt")

In [ ]:
out = model.generate(inpt.cuda(), max_length=10, repetition_penalty=5.0, do_sample=True, top_k=5, top_p=0.95, temperature=0.2)

In [ ]:
out

tensor([[  292,   312,  1923,   282, 17203, 32751,  3236,  9492, 33607, 33204]],
       device='cuda:0')

In [ ]:
tok.decode(out[0])

'хрипы в легких выслушиваются рассеянные сухие'

In [ ]:
text = "жалобы"
inpt = tok.encode(text, return_tensors="pt")

In [ ]:
out = model.generate(inpt.cuda(), max_length=10, repetition_penalty=5.0, do_sample=True, top_k=5, top_p=0.95, temperature=0.2)

In [ ]:
out

tensor([[ 1646,   627, 19143,   322, 11103,  3576,  5296, 14001,  3380,   539]],
       device='cuda:0')

In [ ]:
tok.decode(out[0])

'жалобы больной не предъявляет состояние удовлетворительное'

In [ ]:
text = "бронхиальная"
inpt = tok.encode(text, return_tensors="pt")

In [ ]:
out = model.generate(inpt.cuda(), max_length=10, repetition_penalty=5.0, do_sample=True, top_k=5, top_p=0.95, temperature=0.2)

In [ ]:
out

tensor([[  296,  1368,  1711,  3050, 37847,   429,  2119,   669,   549,  5189]],
       device='cuda:0')

In [ ]:
tok.decode(out[0])

'бронхиальная астма смешаная форма'

In [ ]:
text = "пациент"
inpt = tok.encode(text, return_tensors="pt")

In [ ]:
out = model.generate(inpt.cuda(), max_length=10, repetition_penalty=5.0, do_sample=True, top_k=5, top_p=0.95, temperature=0.2)

In [ ]:
out

tensor([[  513,   377,   645,   343, 10050, 13029,  1979,  7925,   371,  5744]],
       device='cuda:0')

In [ ]:
tok.decode(out[0])

'пациентка отмечает ухудшение состояния при нах'

In [ ]:
!zip -r /content/gpt_2_3_therapy.zip /content/models/therapy_gpt

  adding: content/models/therapy_gpt/ (stored 0%)
  adding: content/models/therapy_gpt/merges.txt (deflated 76%)
  adding: content/models/therapy_gpt/all_results.json (deflated 54%)
  adding: content/models/therapy_gpt/eval_results.json (deflated 41%)
  adding: content/models/therapy_gpt/vocab.json (deflated 74%)
  adding: content/models/therapy_gpt/tokenizer_config.json (deflated 75%)
  adding: content/models/therapy_gpt/training_args.bin (deflated 52%)
  adding: content/models/therapy_gpt/special_tokens_map.json (deflated 80%)
  adding: content/models/therapy_gpt/runs/ (stored 0%)
  adding: content/models/therapy_gpt/runs/Apr10_23-22-09_b01ea1c613e5/ (stored 0%)
  adding: content/models/therapy_gpt/runs/Apr10_23-22-09_b01ea1c613e5/events.out.tfevents.1712795456.b01ea1c613e5.1915.1 (deflated 26%)
  adding: content/models/therapy_gpt/runs/Apr10_23-22-09_b01ea1c613e5/events.out.tfevents.1712791343.b01ea1c613e5.1915.0 (deflated 63%)
  adding: content/models/therapy_gpt/tokenizer.json (de